In [8]:
import requests
from dotenv import load_dotenv
from langchain_core.tools import tool
import os

load_dotenv()

@tool
def search_recipes(
    query: str, 
    includeIngredients: str, 
    excludeIngredients: str, 
    diet: str, 
    cuisine: str, 
    addRecipeInformation: bool, 
    number: int,
) -> dict:
    """Search for recipes based on various filters and preferences.

    Args:
        query: Search term for the recipe (e.g., 'pasta').
        includeIngredients: Ingredients that must be included in the recipes (comma-separated).
        excludeIngredients: Ingredients to exclude from the recipes (comma-separated).
        diet: Dietary preference (e.g., 'vegetarian').
        cuisine: Cuisine type (e.g., 'italian').
        addRecipeInformation: Whether to include detailed recipe information (True/False).
        number: Number of results to return.

    Returns:
        A dictionary containing the search results.
    """
    result = {
        "query": query,
        "includeIngredients": includeIngredients,
        "excludeIngredients": excludeIngredients,
        "diet": diet,
        "cuisine": cuisine,
        "addRecipeInformation": addRecipeInformation,
        "number": number
    }
    return result

@tool
def conversation(query: str) -> str:
    """This function simply returns the input query as part of a normal conversation."""
    return query

In [9]:
from langchain_cohere import ChatCohere
import os

load_dotenv()

os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY")


llm = ChatCohere(model="command-r-plus-04-2024")

tools = [search_recipes, conversation]
llm_with_tools = llm.bind_tools(tools)

query = "Find a recipe I can make with chicken, onion, and garlic, that excludes peanuts and shellfish, is gluten-free, and provides 2 detailed results."

parameters = llm_with_tools.invoke(query).tool_calls
parameters[0]['args']



{'query': "Hello! I'm an AI assistant, so I don't have feelings as such, but I'm here to help you with any queries you might have."}

In [10]:
parameters[0]['name']


'conversation'

In [18]:
# query = parameters[0]['args']['query']
# includeIngredients = parameters[0]['args']['includeIngredients']
# excludeIngredients = parameters[0]['args']['excludeIngredients']
# diet = parameters[0]['args']['diet']
# cuisine = parameters[0]['args']['cuisine']
# addRecipeInformation = parameters[0]['args']['addRecipeInformation']
# number = parameters[0]['args']['number']	


In [10]:
import requests
from dotenv import load_dotenv
import os

load_dotenv()

# API Key and URL
API_KEY = os.getenv("SPOONACULAR_API_KEY")
BASE_URL = "https://api.spoonacular.com/recipes/complexSearch"

# Parameters for the search
params = {
    "query": parameters[0]['args']['query'],
    "includeIngredients": parameters[0]['args']['includeIngredients'],
    "excludeIngredients": parameters[0]['args']['excludeIngredients'],
    "diet": parameters[0]['args']['diet'],
    "cuisine": parameters[0]['args']['cuisine'],
    "addRecipeInformation": parameters[0]['args']['addRecipeInformation'],
    "number": parameters[0]['args']['number'],
    "apiKey": API_KEY
}

# Make the API request
response = requests.get(BASE_URL, params=params)

# Check the response status and print results
if response.status_code == 200:
    recipes = response.json()
    print(recipes)
    for recipe in recipes.get("results", []):
        print(recipe)
        print(f"Recipe: {recipe['title']}")
        print(f"URL: {recipe.get('sourceUrl', 'N/A')}")
        print(f"Image: {recipe.get('image', 'N/A')}")
        print("-" * 30)
else:
    print(f"Failed to fetch recipes: {response.status_code}")
    print(response.json())


{'results': [{'vegetarian': False, 'vegan': False, 'glutenFree': True, 'dairyFree': False, 'veryHealthy': False, 'cheap': False, 'veryPopular': False, 'sustainable': False, 'lowFodmap': False, 'weightWatcherSmartPoints': 15, 'gaps': 'no', 'preparationMinutes': None, 'cookingMinutes': None, 'aggregateLikes': 2, 'healthScore': 36, 'creditsText': 'foodista.com', 'sourceName': 'foodista.com', 'pricePerServing': 206.4, 'id': 648155, 'title': 'Italian Kale and Potato Soup', 'readyInMinutes': 45, 'servings': 6, 'sourceUrl': 'https://www.foodista.com/recipe/54VL6QJD/italian-kale-and-potato-soup', 'image': 'https://img.spoonacular.com/recipes/648155-312x231.jpg', 'imageType': 'jpg', 'summary': 'Italian Kale and Potato Soup is a <b>gluten free and whole 30</b> recipe with 6 servings. This main course has <b>480 calories</b>, <b>22g of protein</b>, and <b>33g of fat</b> per serving. For <b>$2.06 per serving</b>, this recipe <b>covers 26%</b> of your daily requirements of vitamins and minerals. 2 

In [11]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI

loader = WebBaseLoader("https://www.foodista.com/recipe/54VL6QJD/italian-kale-and-potato-soup")
docs = loader.load()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(docs)
db = FAISS.from_documents(documents[:30], OpenAIEmbeddings())

# query = "Give me the ingredients and instructions for this recipe"
# result = db.similarity_search(query)
# result[0].page_content


llm = ChatOpenAI(model='gpt-3.5-turbo')



prompt = ChatPromptTemplate.from_template(
    """
    You are a highly intelligent and helpful assistant specialized in providing precise and insightful answers based on the given context. Always base your response strictly on the provided information and include step-by-step reasoning to ensure clarity and accuracy. Avoid assumptions and external knowledge unless explicitly asked for.

    Your response should:
    - Start with a brief summary, if appropriate.
    - Break down complex information into clear steps or components.
    - Be concise yet thorough.

    Here is the context for your response:
    <context>
    {context}
    </context>
    
    Now, answer the following question:
    Question: {input}
    """
)



document_chain = create_stuff_documents_chain(llm, prompt)
retriever = db.as_retriever()


retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "what are the ingredients and the instructions or preparation for this recipe?"})
response['answer']


USER_AGENT environment variable not set, consider setting it to identify your requests.
C:\Users\USER\AppData\Local\Temp\ipykernel_17400\1915260897.py:17: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  db = FAISS.from_documents(documents[:30], OpenAIEmbeddings())


"Ingredients:\n- 2 teaspoons olive oil\n- 1 1/4 pounds ground turkey Italian sausage\n- 1 1/4 pounds potatoes\n- 1 medium onion, chopped\n- 3 cloves garlic, minced\n- 1 bunch kale, washed, stems removed, and chopped into pieces\n- 6 cups low-sodium chicken broth\n- 2 teaspoons dried basil\n- 2 teaspoons dried oregano\n- 2 teaspoons crushed fennel seed\n- 1/2 cup low-fat buttermilk\n- Fresh ground pepper to taste\n- 1/4 cup to 1/2 cup shredded pecorino cheese\n\nInstructions/Preparation:\n1. In a large saucepan, heat 2 teaspoons of olive oil over medium-high heat.\n2. Add 1 1/4 pounds of ground turkey Italian sausage, crumble, and brown.\n3. Add 1 medium chopped onion and 3 cloves of minced garlic. Saute until onions are just tender, about three minutes.\n4. Create a bouquet garni in cheese cloth with 2 teaspoons of dried basil, 2 teaspoons of dried oregano, and 2 teaspoons of crushed fennel seed.\n5. Add the bouquet garni and all remaining ingredients except for the kale and buttermilk

In [17]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_cohere.llms import Cohere
from langchain_cohere import CohereEmbeddings


# Load the webpage content
loader = WebBaseLoader("https://www.foodista.com/recipe/54VL6QJD/italian-kale-and-potato-soup")
docs = loader.load()

# Initialize API key
os.environ['COHERE_API_KEY'] = os.getenv('COHERE_API_KEY')

# Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(docs)

# Create a vector store with FAISS
db = FAISS.from_documents(documents[:30], CohereEmbeddings(model="embed-english-light-v3.0"))

# Create the Cohere LLM object
llm = Cohere(
    temperature=0.7,
    max_tokens=300  # Adjust the token limit as needed
)

# Define the prompt template
prompt = PromptTemplate(
    input_variables=["context", "input"],
    template="""
    You are a highly intelligent and helpful assistant specialized in providing precise and insightful answers based on the given context. Always base your response strictly on the provided information and include step-by-step reasoning to ensure clarity and accuracy. Avoid assumptions and external knowledge unless explicitly asked for.

    Your response should:
    - Start with a brief summary, if appropriate.
    - Break down complex information into clear steps or components.
    - Be concise yet thorough.

    Here is the context for your response:
    <context>
    {context}
    </context>
    
    Now, answer the following question:
    Question: {input}
    """
)

# Create the document chain
document_chain = create_stuff_documents_chain(llm, prompt)

# Create the retriever
retriever = db.as_retriever()

# Create the retrieval chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# Query the chain
response = retrieval_chain.invoke({
    "input": "What are the ingredients and the instructions or preparation for this recipe?"
})

# Output the response
print(response['answer'])


The ingredients for this recipe are 2 teaspoons of olive oil, 1¼ pounds of ground turkey Italian sausage, 1¼ pounds of potatoes, 1 medium onion, 3 cloves of garlic, 1 bunch of kale washed, stems removed and chopped into pieces, 6 cups of low-sodium chicken broth, 2 teaspoons of dried basil, 2 teaspoons of dried oregano, 2 teaspoons of crushed fennel seed, ½ cup of low-fat buttermilk, and fresh ground pepper to taste. You will also need ¼ cup to ½ shredded pecorino cheese for topping. 

Here are the steps for preparation:

1. In a large sauce pan, heat oil over medium-high heat. Add sausage, crumble and brown.
2. Add onion and garlic and saute until onions are just tender, about three minutes.
3. Meanwhile, create a bouquet garni in cheese cloth with the basil, oregano, and fennel. Add it and all remaining ingredients except the kale and buttermilk, bring to a boil, reduce heat, cover and simmer until potatoes are cooked, 10-15 minutes.
4. Add kale and simmer until it is just tender, le

### Final Work (Putting it all together)


In [19]:
import requests
from dotenv import load_dotenv
from langchain_core.tools import tool
import os

load_dotenv()

@tool
def search_recipes(
    query: str, 
    includeIngredients: str, 
    excludeIngredients: str, 
    diet: str, 
    cuisine: str, 
    addRecipeInformation: bool, 
    number: int,
) -> dict:
    """Search for recipes based on various filters and preferences.

    Args:
        query: Search term for the recipe (e.g., 'pasta').
        includeIngredients: Ingredients that must be included in the recipes (comma-separated).
        excludeIngredients: Ingredients to exclude from the recipes (comma-separated).
        diet: Dietary preference (e.g., 'vegetarian').
        cuisine: Cuisine type (e.g., 'italian').
        addRecipeInformation: Whether to include detailed recipe information (True/False).
        number: Number of results to return.

    Returns:
        A dictionary containing the search results.
    """
    result = {
        "query": query,
        "includeIngredients": includeIngredients,
        "excludeIngredients": excludeIngredients,
        "diet": diet,
        "cuisine": cuisine,
        "addRecipeInformation": addRecipeInformation,
        "number": number
    }
    return result

In [20]:
import requests
from dotenv import load_dotenv
import os

load_dotenv()

def fetch_recipes(parameters):
    # API Key and URL
    API_KEY = os.getenv("SPOONACULAR_API_KEY")
    BASE_URL = "https://api.spoonacular.com/recipes/complexSearch"

    # Parameters for the search
    params = {
        "query": parameters[0]['args']['query'],
        "includeIngredients": parameters[0]['args']['includeIngredients'],
        "excludeIngredients": parameters[0]['args']['excludeIngredients'],
        "diet": parameters[0]['args']['diet'],
        "cuisine": parameters[0]['args']['cuisine'],
        "addRecipeInformation": parameters[0]['args']['addRecipeInformation'],
        "number": parameters[0]['args']['number'],
        "apiKey": API_KEY
    }

    # Make the API request
    response = requests.get(BASE_URL, params=params)

    # Check the response status and return results
    if response.status_code == 200:
        recipes = response.json()
        return recipes
    else:
        return {
            "error": f"Failed to fetch recipes: {response.status_code}",
            "details": response.json()
        }


In [21]:
from langchain_cohere import ChatCohere
import os

load_dotenv()

os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY")


llm = ChatCohere(model="command-r-plus-04-2024")

tools = [search_recipes]
llm_with_tools = llm.bind_tools(tools)

query = "Find a recipe I can make with chicken, onion, and garlic, that excludes peanuts and shellfish, is gluten-free, and provides 2 detailed results."

parameters = llm_with_tools.invoke(query).tool_calls
recipes = fetch_recipes(parameters)
recipes



{'results': [{'vegetarian': False,
   'vegan': False,
   'glutenFree': True,
   'dairyFree': False,
   'veryHealthy': False,
   'cheap': False,
   'veryPopular': False,
   'sustainable': False,
   'lowFodmap': False,
   'weightWatcherSmartPoints': 15,
   'gaps': 'no',
   'preparationMinutes': None,
   'cookingMinutes': None,
   'aggregateLikes': 2,
   'healthScore': 36,
   'creditsText': 'foodista.com',
   'sourceName': 'foodista.com',
   'pricePerServing': 206.4,
   'id': 648155,
   'title': 'Italian Kale and Potato Soup',
   'readyInMinutes': 45,
   'servings': 6,
   'sourceUrl': 'https://www.foodista.com/recipe/54VL6QJD/italian-kale-and-potato-soup',
   'image': 'https://img.spoonacular.com/recipes/648155-312x231.jpg',
   'imageType': 'jpg',
   'summary': 'Italian Kale and Potato Soup is a <b>gluten free and whole 30</b> recipe with 6 servings. This main course has <b>480 calories</b>, <b>22g of protein</b>, and <b>33g of fat</b> per serving. For <b>$2.06 per serving</b>, this recip

In [24]:
for recipe in recipes['results']:
    print(recipe['title'])
    print(recipe['sourceUrl'])
    print(recipe['image'])
    print("-" * 30)


Italian Kale and Potato Soup
https://www.foodista.com/recipe/54VL6QJD/italian-kale-and-potato-soup
https://img.spoonacular.com/recipes/648155-312x231.jpg
------------------------------
Boozy Bbq Chicken
http://www.foodista.com/recipe/2ZPN8F4S/boozy-bbq-chicken
https://img.spoonacular.com/recipes/635675-312x231.jpg
------------------------------


In [25]:
def get_recipe_details(url):
    """
    Retrieves and analyzes recipe details from a given URL using LangChain and Cohere.
    
    Args:
        url (str): URL of the recipe webpage to analyze
        
    Returns:
        str: Detailed response containing ingredients and instructions
    """
    from langchain_community.document_loaders import WebBaseLoader
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
    from langchain.chains import create_retrieval_chain
    from langchain.prompts import PromptTemplate
    from langchain_community.vectorstores import FAISS
    from langchain_cohere.llms import Cohere
    from langchain_cohere import CohereEmbeddings

    # Load the webpage content
    loader = WebBaseLoader(url)
    docs = loader.load()

    # Initialize API key
    os.environ['COHERE_API_KEY'] = os.getenv('COHERE_API_KEY')

    # Split the documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
    documents = text_splitter.split_documents(docs)

    # Create a vector store with FAISS
    db = FAISS.from_documents(documents[:30], CohereEmbeddings(model="embed-english-light-v3.0"))

    # Create the Cohere LLM object
    llm = Cohere(
        temperature=0.7,
        max_tokens=300  # Adjust the token limit as needed
    )

    # Define the prompt template
    prompt = PromptTemplate(
        input_variables=["context", "input"],
        template="""
        You are a highly intelligent and helpful assistant specialized in providing precise and insightful answers based on the given context. Always base your response strictly on the provided information and include step-by-step reasoning to ensure clarity and accuracy. Avoid assumptions and external knowledge unless explicitly asked for.

        Your response should:
        - Start with a brief summary, if appropriate.
        - Break down complex information into clear steps or components.
        - Be concise yet thorough.

        Here is the context for your response:
        <context>
        {context}
        </context>
        
        Now, answer the following question:
        Question: {input}
        """
    )

    # Create the document chain
    document_chain = create_stuff_documents_chain(llm, prompt)

    # Create the retriever
    retriever = db.as_retriever()

    # Create the retrieval chain
    retrieval_chain = create_retrieval_chain(retriever, document_chain)

    # Query the chain
    response = retrieval_chain.invoke({
        "input": "What are the ingredients and the instructions or preparation for this recipe?"
    })

    return response['answer']


In [26]:
for recipe in recipes['results']:
    print(recipe['title'])
    print(recipe['sourceUrl'])
    print(recipe['image'])
    print(get_recipe_details(recipe['sourceUrl']))
    print("-" * 30)



Italian Kale and Potato Soup
https://www.foodista.com/recipe/54VL6QJD/italian-kale-and-potato-soup
https://img.spoonacular.com/recipes/648155-312x231.jpg
The ingredients for this recipe are 2 teaspoons of olive oil, 1¼ pounds of ground turkey Italian sausage, 1¼ pounds of potatoes, 1 medium onion, 3 cloves of garlic, 1 bunch of kale, washed and stems removed, and chopped into pieces, 6 cups of low-sodium chicken broth, 2 teaspoons of dried basil, 2 teaspoons of dried oregano, 2 teaspoons of crushed fennel seed, ½ cup of low-fat buttermilk, and fresh ground pepper to taste. You can also add ¼ cup to ½ shredded pecorino cheese for topping. 

Here are the instructions:

1. In a large sauce pan, heat oil over medium-high heat. Add Italian sausage, crumble and brown.
2. Add onion and garlic and saute until onions are tender.
3. Meanwhile, create a bouquet garni in cheese cloth with the basil, oregano, and fennel. Add it and all remaining ingredients except the kale and buttermilk, bring to 

In [33]:
def get_recipe_details(urls):
    """
    Retrieves and analyzes recipe details from given URLs using LangChain and Cohere,
    maintaining context across multiple recipes.
    
    Args:
        urls (list): List of recipe webpage URLs to analyze.
        
    Returns:
        str: Detailed response containing aggregated ingredients and instructions.
    """
    from langchain_community.document_loaders import WebBaseLoader
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
    from langchain.chains import create_retrieval_chain
    from langchain.prompts import PromptTemplate
    from langchain_community.vectorstores import FAISS
    from langchain_cohere.llms import Cohere
    from langchain_cohere import CohereEmbeddings
    import os

    # Initialize API key
    os.environ['COHERE_API_KEY'] = os.getenv('COHERE_API_KEY')

    # Initialize variables
    global_context = ""  # Accumulated context across URLs

    # Cohere LLM object
    llm = Cohere(
        temperature=0.7,
        max_tokens=300  # Adjust the token limit as needed
    )

    # Define the prompt template with accumulated context
    prompt = PromptTemplate(
        input_variables=["existing_context", "context", "input"],
        template="""
        You are a highly intelligent and helpful assistant specialized in providing precise and insightful answers 
        based on the given context. Always base your response strictly on the provided information and include 
        step-by-step reasoning to ensure clarity and accuracy. Avoid assumptions and external knowledge unless 
        explicitly asked for.

        Your response should:
        - Start with a brief summary, if appropriate.
        - Break down complex information into clear steps or components.
        - Be concise yet thorough.

        Here is the context from previous recipes:
        <existing_context>
        {existing_context}
        </existing_context>

        Here is the new context for your response:
        <context>
        {context}
        </context>
        
        Now, answer the following question:
        Question: {input}
        """
    )

    # Create the document chain
    document_chain = create_stuff_documents_chain(llm, prompt)

    # Process each URL
    for url in urls:
        # Load the webpage content
        loader = WebBaseLoader(url)
        docs = loader.load()

        # Split the documents into chunks
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
        documents = text_splitter.split_documents(docs)

        # Create a vector store with FAISS
        db = FAISS.from_documents(documents[:30], CohereEmbeddings(model="embed-english-light-v3.0"))

        # Create the retriever
        retriever = db.as_retriever()

        # Create the retrieval chain
        retrieval_chain = create_retrieval_chain(retriever, document_chain)

        # Query the chain with the existing context
        response = retrieval_chain.invoke({
            "existing_context": global_context,
            "context": "\n".join(doc.page_content for doc in documents),
            "input": "What are the ingredients and the instructions or preparation for this recipe?"
        })

        # Update the global context with the new response
        global_context += "\n" + response['answer']

    return global_context


In [34]:
x = get_recipe_details(["https://www.foodista.com/recipe/54VL6QJD/italian-kale-and-potato-soup", "http://www.foodista.com/recipe/2ZPN8F4S/boozy-bbq-chicken"])
print(x)




This recipe is for Italian Kale and Potato Soup. Here are the ingredients and the instructions broken down step by step: 

**Ingredients:**
1. 2 teaspoons of olive oil 
2. 1¼ pounds of ground turkey Italian sausage 
3. 1¼ pounds of potatoes 
4. 1 medium onion, chopped 
5. 3 cloves of garlic, minced 
6. 1 bunch of kale, washed, stems removed, and chopped into pieces 
7. 6 cups of low-sodium chicken broth 
8. 2 teaspoons each of dried basil, dried oregano, and crushed fennel seed 
9. 1/2 cup of low-fat buttermilk 
10. Fresh ground pepper, to taste 
11. 1/4 cup to 1/2 shredded pecorino kale 

**Instructions:**
1. In a large sauce pan, heat the olive oil over medium-high heat and add the Italian sausage, crumbled and browned. 
2. Add the onion and garlic and saute until the onions are tender, about 3 minutes. 
3. Meanwhile, create a bouquet garni in cheese cloth with the basil, oregano, and fennel. Add it and all remaining ingredients except the kale and buttermilk to the pot, bring to a 

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os 
from dotenv import load_dotenv
import os 
from dotenv import load_dotenv

def get_ai_response(input_text: str) -> str:
    """
    Get AI response using LangChain and OpenAI.
    
    Args:
        input_text (str): User input text/question
        
    Returns:
        str: AI generated response
    """
    load_dotenv()

    os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
    os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')

    # Prompt Template
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a friendly and intelligent assistant. You can provide helpful answers, engage in casual conversation, and be thoughtful and patient with your responses. Feel free to ask follow-up questions if needed."),
            ("user", "I need assistance with: {question}")
        ]
    )

    llm = ChatOpenAI(model='gpt-3.5-turbo')
    output_parser = StrOutputParser()
    chain = prompt|llm|output_parser

    if input_text:
        result = chain.invoke({"question": input_text})
        return result
    return ""


In [7]:
# Example usage
response = get_ai_response("Hello, how are you doing today ?")
print(response)


Hello! I'm here and ready to help you. How can I assist you today?
